# Stage 1 - Data Cleaning and Manipulation with Pandas

For this stage, you will start with this messy dataset Forbes Billionaires 2018. You will need to import it, use your data cleaning skills to clean it up, prepare it to be analyzed, and then export it as a clean CSV data file. Here `http://potacho.com/files/ironhack/<your-kaggle-name>.db` you may find the `.db` file with the raw dataset. 

Deliverables:
- Produce at least one Jupyter Notebook that shows the steps you took and the code you used to acquire, wrangle (clean and transform), analyze and report your data.
- Produce at least one processed data file (any format) that you will use in upcoming stages.

Suggested tools: pandas, pathlib, sqlalchemy, matplotlib, seaborn, plotly

In [1]:
#Import pandas and sqlite3
import pandas as pd
import numpy as np
import sqlite3
import re
import os
import matplotlib.pyplot as plt

In [2]:
#Create a conection to bbdd throught SQLite
conex = sqlite3.connect("/Users/borjauria/IRONHACK/Ironhack-Module-1-Project---The-best-ever-project/data/raw/borjauria.db")

#Extracts the query data directly to a DataFrame
personal_info = pd.read_sql_query("SELECT * from personal_info", conex)
business_info = pd.read_sql_query("SELECT * from business_info", conex)
rank_info = pd.read_sql_query("SELECT * from rank_info", conex)

In [3]:
#Preview of data
df_outer = pd.merge(personal_info, business_info, on='id', how='outer')
df = pd.merge(df_outer, rank_info, on='id', how='outer')

In [4]:
#I change the name of the rows to know
df.rename(columns={'lastName':'Last Name', 'age': 'Age', 'Unnamed: 0': 'Unnamed', 'gender': 'Gender', 'country': 'From', 'image': 'Photo', 'name': 'Name'}, inplace=True)

In [5]:
# Replace Values
df['Age'] = df['Age'].str.replace(r"[a-zA-Z]+",'')
df['From'] = df['From'].str.replace(r"USA|USA",'United States')
df['Name'] = df['Name'].str.replace(r"( \S+)",'')
df['Gender'] = df['Gender'].str.replace(r"[M]",'Male')
df['Gender'] = df['Gender'].str.replace(r"Maleale",'Male')
df['Gender'] = df['Gender'].str.replace(r"(^F$)",'Female')
df['worth'] = df['worth'].str.replace(r"[a-zA-Z]+",'')
df['worthChange'] = df['worthChange'].str.replace(r"[a-zA-Z]+",'')

In [6]:
# Convert all inside of "Last Name" in lowercase.
df['Last Name'] = df['Last Name'].str.capitalize()

In [7]:
df.rename(columns={'Unnamed: 0': 'Unnamed', 'Source': 'Source & company', 'worthChange': 'Worth Change', 'realTimeWorth': 'Real Time Worth', 'realTimePosition': 'Real Time Position'}, inplace=True)

In [8]:
# I create a new data frame without 'Real Time Worth column, Unnamed: 0_x, Unnamed: 0_y, Unnamed'.
df = df.drop('Real Time Worth', 1)
df = df.drop('Unnamed: 0_x', 1)
df = df.drop('Unnamed: 0_y', 1)
df = df.drop('Unnamed', 1)

In [9]:
#New data frame with split value columns from Source & Company
df[['Industry','Company']] = df["Source & company"].str.split("==> ",expand=True)

In [10]:
#New data frame without the 'Source & company' column.
df = df.drop('Source & company', 1)

In [11]:
df['Name'] = df['Name'].str.capitalize()

In [12]:
df['Last Name'].fillna('none', inplace=True)
df['Gender'].fillna('', inplace=True)
df['From'].fillna('none', inplace=True)
df['Photo'].fillna('none', inplace=True)
df['worth'].fillna('none', inplace=True)
df['Worth Change'].fillna('none', inplace=True)
df['Real Time Position'].fillna('none', inplace=True)
df['position'].fillna('none', inplace=True)
df['Industry'].fillna('none', inplace=True)
df['Company'].fillna('none', inplace=True)

In [13]:
# Here I change the type of the data
df['Age'].replace(to_replace='None',value=-9999, inplace=True)
df['Gender'].replace(to_replace='None', inplace=True)
df['From'].replace(to_replace='None', inplace=True)
df['Age'].fillna(-9999, inplace=True)
df = df.astype({'Age':int})
df.loc[df['Age']> 100, 'Age'] = 2018 - df['Age']

In [14]:
df['Gender'].replace('', np.nan, inplace=True)
df['From'].replace('', np.nan, inplace=True)

In [15]:
df = df.dropna()

In [16]:
df['Name'] = df['Name'] +' '+ df['Last Name']

In [17]:
df = df[['id','Name','Age','Gender','From', 'Industry','Company','worth','Worth Change','Real Time Position','Photo']]

In [18]:
# Saving the dataframe 
df.to_csv(r'/Users/borjauria/IRONHACK/Ironhack-Module-1-Project---The-best-ever-project/data/processed/borjauria_done.csv', index=False) 